In [2]:
import reservoirpy as rpy
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)
import numpy as np
import math
import pandas as pd
from functools import partial
import sys
import os
import time
import matplotlib.pyplot as plt
from memory_profiler import memory_usage

# Add the parent directory to the sys.path list
sys.path.append(os.path.abspath('../'))
import NAS.NAS
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

rpy.verbosity(0)
output_dim = 1

In [3]:
def nrmseGESN(yTrue, preds):
    norm = 0
    mse = 0
    targetMean = np.mean(yTrue)
    for i, target in enumerate(yTrue):
        norm+=(target - targetMean)**2
        mse+=(target - preds[i])**2
    return math.sqrt(mse/norm)

def nmse(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    sigma2 = np.var(y)  # variance of y
    i = len(y)
    error = sum((o[t] - y[t])**2 for t in range(i))
    return error / (i * sigma2)

def mae(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    i = len(y)
    error = sum(abs(o[t] - y[t]) for t in range(i))
    return error / i

def mape(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    i = len(y)
    error = sum(abs(o[t] - y[t]) / abs(y[t]) for t in range(i))
    return (error / i) * 100

def nrmseDBESN(yTrue, preds):
    norm = 0
    mse = 0
    targetMean = np.mean(yTrue)
    for i, target in enumerate(yTrue):
        norm+=(target - targetMean)**2
        mse+=(target - preds[i])**2
    return math.sqrt(mse/norm)

def r_squared(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    numerator = np.sum((y_true - y_pred)**2)
    denominator = np.sum((y_true - np.mean(y_true))**2)
    
    return 1 - (numerator / denominator)


In [3]:
# https://www.sciencedirect.com/science/article/pii/S0925231222014291
# Parameterizing echo state networks for multi-step time series prediction
# Mackey glass dataset

def getData():
    data = np.load('../data/MG17.npy')
    data = data.reshape((data.shape[0],1))
    data = data[:3801,:]
    from scipy import stats
    data = stats.zscore(data)
    data.shape

    trainLen = 1100
    valLen = 286
    testLen = 286
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(NAS.NAS.evaluateArchitecture, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.NAS.generateRandomArchitecture, sampleX=trainX[:2], sampleY=trainY[:2]),
    "populationSize": 15,
    "eliteSize": 1,
    "stagnationReset": 5,
    "generations": 20,
    "minimizeFitness": True,
    "logModels": False,
    "seedModels": [
        # {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2048, 'input_connectivity': 0.44, 'rc_connectivity': 0.44, 'fb_connectivity': 0.44, 'sr': 1.406, 'lr': 0.68}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.0e-7}}, ], 'edges': [[0, 1], [1, 2]]}
    ],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0,
    "n_jobs": 5
}

nrmseErrors = []
r2Errors = []
for i in range(1):
    models, performances, architectures = NAS.NAS.runGA(gaParams)
    allPreds = []
    for architecture in architectures[:5]:
        for _ in range(10):
            model = NAS.NAS.constructModel(architecture)
            model = NAS.NAS.trainModel(model, np.concatenate([trainX, valX[:-20]]), np.concatenate([trainY, valY[:-20]]))
            prevOutput = valX[-20]
            preds = []
            for j in range(20+len(testX)):
                pred = NAS.NAS.runModel(model, prevOutput)
                prevOutput = pred
                preds.append(pred[0])
            preds = np.array(preds)
            allPreds.append(preds)
    valErrors = []
    for pred in allPreds:
        valError = nrmse(valY[-20:], pred[:20])
        error = nrmse(testY, pred[-len(testY):])
        valErrors.append(valError)
    bestPred = allPreds[valErrors.index(min(valErrors))]
    print(performances[0:5], min(valErrors), nrmse(testY, bestPred[-len(testY):]), r_squared(testY, bestPred[-len(testY):]))
    if min(valErrors)>0.0003:
        continue
    nrmseErrors.append(nrmse(testY, bestPred[-len(testY):]))
    r2Errors.append(r_squared(testY, bestPred[-len(testY):]))

print(np.array(nrmseErrors).mean(), np.array(nrmseErrors).std())
print(np.array(r2Errors).mean(), np.array(r2Errors).std())

[0.228129311960515, 0.3660608203150648, 0.5342405279865889, 0.5821800327187896, 0.8017676653301289] 0.00013600086395423744 0.13077302810030897
0.13077302810030897 0.0
0.9999060198302299 0.0


In [16]:
def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_norm = np.linalg.norm(np.mean(y_true))
    
    return rmse / mean_norm

for i in range(10):
    model = NAS.NAS.constructModel({'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2048, 'input_connectivity': 0.44, 'rc_connectivity': 0.44, 'fb_connectivity': 0.44, 'sr': 1.406, 'lr': 0.68}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.0e-7}}, ], 'edges': [[0, 1], [1, 2]]})
    model = NAS.NAS.trainModel(model, trainX, trainY)
    prevOutput = valX[0]
    preds = []
    for _ in range(len(valX)):
        pred = NAS.NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    print("Performance", nrmse(valY, preds), r_squared(valY, preds))

Performance 0.17397043066592893 0.9999842165217776
Performance 0.2553627231389931 0.9999659931159932
Performance 0.1709178220686728 0.9999847655584622
Performance 0.5984152053128649 0.9998132515187916
Performance 0.4950672143867095 0.9998721854866306
Performance 0.035495457408136005 0.9999993429515542
Performance 0.06543405711252026 0.9999977671500669
Performance 0.34416378398684705 0.9999382293495855
Performance 0.02668590016927505 0.9999996286225309
Performance 0.7394688258406639 0.9997148380485067


In [18]:
for i in range(20):
    model = NAS.NAS.constructModel({'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2048, 'input_connectivity': 0.44, 'rc_connectivity': 0.44, 'fb_connectivity': 0.44, 'sr': 1.406, 'lr': 0.68}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.0e-7}}, ], 'edges': [[0, 1], [1, 2]]})
    model = NAS.NAS.trainModel(model, np.concatenate([trainX, valX]), np.concatenate([trainY, valY]))
    prevOutput = testX[0]
    preds = []
    for _ in range(len(testX)):
        pred = NAS.NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    print("Performance", nrmse(testY, preds), r_squared(testY, preds))

Performance 0.8197699574638088 0.9999665454958991
Performance 0.9325962319625812 0.9999567029893367
Performance 0.6472014252382464 0.999979147912928
Performance 0.47078184739200785 0.9999889665833812
Performance 0.6666787215181004 0.999977873955191
Performance 0.3656201840379366 0.9999933452625542
Performance 0.40109330049621555 0.999991991311247
Performance 2.4894700110165076 0.9996914792362921
Performance 0.6932497321979203 0.9999760751060864
Performance 0.773611777612678 0.9999702068278821
Performance 0.8106088979086535 0.999967289036761
Performance 0.5315187788434534 0.9999859360334196
Performance 1.406469109156257 0.9999015238648233
Performance 0.6578099445118061 0.9999784587218209
Performance 0.4287678483050176 0.9999908480208226
Performance 0.16962681610715463 0.999998567615474
Performance 0.6167583427002827 0.9999810634584418
Performance 0.7629774594169056 0.9999710202913195
Performance 0.5910059614904216 0.9999826118121626
Performance 0.6049005760816506 0.9999817846048888


In [22]:
def getData():
    data = np.load('../data/Lorenz_normed_2801.npy')
    # data = data[:3801,:]
    from scipy import stats
    data = stats.zscore(data)

    trainLen = 2000
    valLen = 444
    testLen = 286
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out
    
trainX, trainY, valX, valY, testX, testY = getData()

def getData():
    sunspots = pd.read_csv("../data/santafelaser.csv")
    data = np.array(sunspots)
    data = data.reshape((data.shape[0],1))
    data = data[:3801,:]
    from scipy import stats
    data = stats.zscore(data)

    trainLen = 2000
    valLen = 100
    testLen = 100
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

# def getData():
#     data = np.load('../data/Neutral_normed_2801.npy')
#     # data = data[:3801,:]
#     from scipy import stats
#     data = stats.zscore(data)

#     trainLen = 2000
#     valLen = 500
#     testLen = 286
#     train_in = data[0:trainLen]
#     train_out = data[0+1:trainLen+1]
#     val_in = data[trainLen:trainLen+valLen]
#     val_out = data[trainLen+1:trainLen+valLen+1]
#     test_in = data[trainLen+valLen:trainLen+valLen+testLen]
#     test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
#     return train_in, train_out, val_in, val_out, test_in, test_out
    
# trainX, trainY, valX, valY, testX, testY = getData()

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_norm = np.linalg.norm(np.mean(y_true))
    
    return rmse / mean_norm

for _ in range(20):
    # architecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2048, 'rc_connectivity': 0, 'sr': 0.995, 'lr': 0}}, {'type': 'Ridge', 'params': {'output_dim': trainX.shape[1], 'ridge': 1.0e-6}}, ], 'edges': [[0, 1], [1, 2]]}
    architecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 1024, 'input_connectivity': 0.84, 'rc_connectivity': 0.84, 'sr': 0.906, 'lr': 0.41}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 8.1e-7}}, ], 'edges': [[0, 1], [1, 2]]}
    # architecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 256, 'rc_connectivity': 0.91, 'sr': 0.995, 'lr': 0.84}}, {'type': 'Ridge', 'params': {'output_dim': trainX.shape[1], 'ridge': 8.3e-7}}, ], 'edges': [[0, 1], [1, 2]]}
    model = NAS.NAS.constructModel(architecture)
    model = NAS.NAS.trainModel(model, trainX, trainY)
    # error1, _, _ = NAS.NAS.evaluateArchitecture2(architecture, trainX, trainY, valX, valY, 1)
    # print(error1)
    # preds = NAS.NAS.runModel(model, valX)
    prevOutput = valX[0]
    preds = []
    for _ in range(len(valX)):
        pred = NAS.NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    print(nrmse(valY, preds), r_squared(valY, preds))

    # plt.plot(preds)
    # plt.plot(valY)
    # plt.show()

6.680105447642031 0.9951289717598434
7.675773893842853 0.9935687060824732
13.195400947480467 0.9809936293935397
2.7468527844286648 0.9991763839900447
4.629128590767261 0.9976608800040913
10.038428676203418 0.9890001823241212
9.73976455318868 0.9896449803262252
6.3369404706889 0.9956165781697571
9.047307148789457 0.9910650386041536
9.592494605236297 0.9899557586737955


KeyboardInterrupt: 

In [20]:
trainX.shape

(2000, 1)

In [46]:
def getData():
    data = np.load('../data/Neutral_normed_2801.npy')
    data = data.reshape((data.shape[0]*data.shape[1],1))
    data = data[:3801,:]
    from scipy import stats
    data = stats.zscore(data)
    data.shape

    trainLen = 2000
    valLen = 500
    testLen = 286
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out
    
trainX, trainY, valX, valY, testX, testY = getData()

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_norm = np.linalg.norm(np.mean(y_true))
    
    return rmse / mean_norm

architecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 256, 'rc_connectivity': 0.91, 'sr': 0.995, 'lr': 0.84}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 8.3e-7}}, ], 'edges': [[0, 1], [1, 2]]}
model = NAS.NAS.constructModel(architecture)
model = NAS.NAS.trainModel(model, trainX, trainY)
preds = NAS.NAS.runModel(model, valX)
# prevOutput = valX[0]
# preds = []
# for _ in range(len(valX)):
#     pred = NAS.NAS.runModel(model, prevOutput)
#     prevOutput = pred
#     preds.append(pred[0])
# preds = np.array(preds)
print(nrmse(valY, preds))


plt.plot(preds)
plt.plot(valY)
plt.show()

0.0029074618392706834


In [3]:
# https://www.sciencedirect.com/science/article/pii/S0925231222014291
# Parameterizing echo state networks for multi-step time series prediction
# Santafe laser dataset

def getData():
    sunspots = pd.read_csv("../data/santafelaser.csv")
    data = np.array(sunspots)

    trainLen = 9891
    valLen = 100
    testLen = 100
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(NAS.NAS.evaluateArchitecture, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.NAS.generateRandomArchitecture, sampleX=trainX[:2], sampleY=trainY[:2]),
    "populationSize": 15,
    "eliteSize": 1,
    "stagnationReset": 5,
    "generations": 10,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [
        # {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2048, 'input_connectivity': 0.44, 'rc_connectivity': 0.44, 'fb_connectivity': 0.44, 'sr': 1.406, 'lr': 0.68}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.0e-7}}, ], 'edges': [[0, 1], [1, 2]]}
    ],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0,
    "n_jobs": 5
}

nrmseErrors = []
r2Errors = []
for i in range(1):
    models, performances, architectures = NAS.NAS.runGA(gaParams)
    allPreds = []
    for architecture in architectures[:5]:
        for _ in range(10):
            model = NAS.NAS.constructModel(architecture)
            model = NAS.NAS.trainModel(model, np.concatenate([trainX, valX[:-20]]), np.concatenate([trainY, valY[:-20]]))
            prevOutput = valX[-20]
            preds = []
            for j in range(20+len(testX)):
                pred = NAS.NAS.runModel(model, prevOutput)
                prevOutput = pred
                preds.append(pred[0])
            preds = np.array(preds)
            allPreds.append(preds)
    valErrors = []
    for pred in allPreds:
        valError = nrmse(valY[-20:], pred[:20])
        error = nrmse(testY, pred[-len(testY):])
        valErrors.append(valError)
    bestPred = allPreds[valErrors.index(min(valErrors))]
    print(performances[0:5], min(valErrors), nrmse(testY, bestPred[-len(testY):]))
    # if min(valErrors)>0.0003:
    #     continue
    nrmseErrors.append(nrmse(testY, bestPred[-len(testY):]))
    r2Errors.append(r_squared(testY, bestPred[-len(testY):]))

print(np.array(nrmseErrors).mean(), np.array(nrmseErrors).std())
print(np.array(r2Errors).mean(), np.array(r2Errors).std())

inf {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'NVAR', 'params': {'delay': 1, 'order': 1, 'strides': 1}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 7.566465540941455e-05}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.44102428592926246}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.025376598082588e-06}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 9.9584953584808e-05}}], 'edges': [[0, 1], [1, 2], [1, 3], [1, 4], [2, 5], [3, 5], [4, 5]]}
0.9330394903442772 {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 2.2694320815997462e-05}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 2.8215816822639563e-05}}, {'type': 'IPReservoir', 'params': {'units': 1251, 'lr': 0.4596392742184165, 'sr': 0.6840221351887465, 'mu': -0.13360157786954296, 'sigma': 1.0226518500237052, 'learning_rate': 0.00297281604213538, 'input_connectivity': 0.16589004713579905, 'rc_connectivity': 0.2850336866745172

In [1]:

def getData():
    sunspots = pd.read_csv("../data/santafelaser.csv")
    data = np.array(sunspots)

    trainLen = 9891
    valLen = 100
    testLen = 100
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

NameError: name 'pd' is not defined

In [4]:
goodArchitecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 1}}, {'type': 'Reservoir', 'params': {'units': 649, 'lr': 0.5975855910585006, 'sr': 1.289221609815766, 'input_connectivity': 0.40010488298206637, 'rc_connectivity': 0.4090786446967834, 'fb_connectivity': 0.09404210228238652}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 2}}, {'type': 'IPReservoir', 'params': {'units': 234, 'lr': 0.9203822465220324, 'sr': 0.5177516030001565, 'mu': 0.0661244738966123, 'sigma': 1.4035618806583001, 'learning_rate': 0.00327840184422856, 'input_connectivity': 0.14644358488436113, 'rc_connectivity': 0.3713680820702342, 'fb_connectivity': 0.273826121341471}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 2}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.045376465443952e-05}}], 'edges': [[0, 1], [0, 2], [0, 3], [1, 4], [3, 5], [2, 6], [4, 6], [5, 6]]}

model = NAS.NAS.constructModel(goodArchitecture)
model = NAS.NAS.trainModel(model, np.concatenate([trainX, valX[:-20]]), np.concatenate([trainY, valY[:-20]]))
prevOutput = valX[-20]
preds = []
for j in range(20+len(testX)):
    pred = NAS.NAS.runModel(model, prevOutput)
    prevOutput = pred
    preds.append(pred[0])

print(nrmse(testY, preds[-len(testY):]), r_squared(testY, preds[-len(testY):]))

0.03571277158613166 0.9897931229697442


In [9]:
# https://www.sciencedirect.com/science/article/pii/S0925231222014291
# Parameterizing echo state networks for multi-step time series prediction
# Neutral normed dataset

def getData():
    data = np.load('../data/Neutral_normed_2801.npy')

    trainLen = 1901
    valLen = 399
    testLen = 500
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(evaluate, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.generateRandomArchitecture, sampleX=trainX[:100], sampleY=trainY[:100]),
    "populationSize": 15,
    "eliteSize": 2,
    "stagnationReset": 5,
    "generations": 22,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0
}

nrmseErrors = []
r2Errors = []
for i in range(1):
    models, performances, architectures = NAS.runGA(gaParams)
    model = models[0]
    startInput = testX[0]
    prevOutput = testX[0]
    preds = []
    for j in range(len(testX)):
        pred = NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    performance = r_squared(testY, preds)
    print("Performance", performance, nrmse(testY, preds))
    nrmseErrors.append(nrmse(testY, preds))
    r2Errors.append(performance)

print(np.array(nrmseErrors).mean(), np.array(nrmseErrors).std())
print(np.array(r2Errors).mean(), np.array(r2Errors).std())

Performance 0.999958227868178 7.311027348144948
Performance 0.9995362054529383 22.118367018388145
Performance -1.2067514774276298 2696.8280088904735
Performance -7.385636140212117 2739.3385181107597
Performance 0.9999629568174395 6.1284839531925925
Performance 0.9804485629026151 137.29728648428116
Performance 0.9999843941124462 4.420965116672592
Performance 0.9999883553787924 4.301596708526212
Performance 0.9999590021977057 6.849569508367799
Performance 0.9853479619681731 119.301864090258
574.3895687229065


In [6]:
# https://www.sciencedirect.com/science/article/pii/S0925231222014291
# Parameterizing echo state networks for multi-step time series prediction
# Lorenz dataset

def getData():
    data = np.load('../data/Lorenz_normed_2801.npy')

    trainLen = 1957
    valLen = 399
    testLen = 444
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(evaluate, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.generateRandomArchitecture, sampleX=trainX[:100], sampleY=trainY[:100]),
    "populationSize": 15,
    "eliteSize": 2,
    "stagnationReset": 5,
    "generations": 22,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0
}

nrmseErrors = []
r2Errors = []
for i in range(1):
    models, performances, architectures = NAS.runGA(gaParams)
    model = models[0]
    startInput = testX[0]
    prevOutput = testX[0]
    preds = []
    for j in range(len(testX)):
        pred = NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    performance = r_squared(testY, preds)
    print("Performance", performance, nrmse(testY, preds))
    nrmseErrors.append(nrmse(testY, preds))
    r2Errors.append(performance)

print(np.array(nrmseErrors).mean(), np.array(nrmseErrors).std())
print(np.array(r2Errors).mean(), np.array(r2Errors).std())


ValueError: Ridge-10is  expecting data of shape (6,) but received shape (3,).

In [ ]:
def getLorenzData():
    data = np.array(lorenz(n_timesteps=10000, rho=28, sigma=10, beta=8/3, x0=[1, 1, 0]))
    train = data[:4000]
    val = data[4000:8000]
    test = data[8000:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY

def getMackeyData():
    data = np.array(mackey_glass(n_timesteps=10000, ))
    train = data[:4000]
    val = data[4000:8000]
    test = data[8000:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY

In [3]:
def getData():
    sunspots = pd.read_csv("../datasets/Sunspots.csv")
    data = sunspots.loc[:,"Monthly Mean Total Sunspot Number"]
    data1 = np.expand_dims(data[:-20], axis=1)
    data2 = np.expand_dims(data[10:-10], axis=1)
    data3 = np.expand_dims(data[20:], axis=1)
    y = data3[1:]
    x = np.concatenate([data1, data2, data3], axis=-1)[:-1]

    trainX = x[:1600]
    trainY = y[:1600]
    valX = x[1600:2100]
    valY = y[1600:2100]
    testX = x[2100:3174]
    testY = y[2100:3174]
    return trainX, trainY, valX, valY, testX, testY

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(evaluate, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.generateRandomArchitecture, sampleX=trainX[:100], sampleY=trainY[:100]),
    "populationSize": 15,
    "eliteSize": 3,
    "stagnationReset": 5,
    "generations": 5,
    "minimizeFitness": True,
    "logModels": False,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0
}

errors = []
for i in range(20):
    models, performances, architectures = NAS.runGA(gaParams)
    model = models[0]
    preds = NAS.runModel(model, testX)
    performance = nrmseGESN(testY, preds)
    print("Performance", performance)
    errors.append(performance)

print(np.array(errors).mean(), np.array(errors).std())

In [5]:
def getData():
    data = np.array(narma(n_timesteps=10000, order=10, a1=0.3, a2=0.05, b=1.5, c=0.1))
    train = data[:4000]
    val = data[4000:8000]
    test = data[8000:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY
trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(NAS.NAS.evaluateArchitecture, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.NAS.generateRandomArchitecture, sampleX=trainX[:2], sampleY=trainY[:2]),
    "populationSize": 15,
    "eliteSize": 1,
    "stagnationReset": 5,
    "generations": 10,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0,
    "n_jobs": 5
}

nmseErrors = []
maeErrors = []
mapeErrors = []
for i in range(1):
    models, performances, architectures = NAS.NAS.runGA(gaParams)
    model = models[0]
    preds = NAS.NAS.runModel(model, testX)
    
    performance = nmse(testY, preds)
    print("Performance", nmse(testY, preds), mae(testY, preds), mape(testY, preds))
    nmseErrors.append(nmse(testY, preds))
    maeErrors.append(mae(testY, preds))
    mapeErrors.append(mape(testY, preds))

print(np.array(nmseErrors).mean(), np.array(nmseErrors).std())
print(np.array(maeErrors).mean(), np.array(maeErrors).std())
print(np.array(mapeErrors).mean(), np.array(mapeErrors).std())

In [6]:
def getData():
    sunspots = pd.read_csv("../datasets/Sunspots.csv")
    data = sunspots.loc[:,"Monthly Mean Total Sunspot Number"]
    data = np.expand_dims(data, axis=1)
    train = data[:2000]
    val = data[2000:2500]
    test = data[2500:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY
trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(NAS.NAS.evaluateArchitecture, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.NAS.generateRandomArchitecture, sampleX=trainX[:2], sampleY=trainY[:2]),
    "populationSize": 15,
    "eliteSize": 1,
    "stagnationReset": 5,
    "generations": 10,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0,
    "n_jobs": 5
}

nmseErrors = []
maeErrors = []
mapeErrors = []
for i in range(1):
    models, performances, architectures = NAS.NAS.runGA(gaParams)
    model = models[0]
    preds = NAS.NAS.runModel(model, testX)
    
    performance = nmse(testY, preds)
    print("Performance", nmse(testY, preds), mae(testY, preds), mape(testY, preds))
    nmseErrors.append(nmse(testY, preds))
    maeErrors.append(mae(testY, preds))
    mapeErrors.append(mape(testY, preds))

print(np.array(nmseErrors).mean(), np.array(nmseErrors).std())
print(np.array(maeErrors).mean(), np.array(maeErrors).std())
print(np.array(mapeErrors).mean(), np.array(mapeErrors).std())

inf {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2096, 'lr': 0.7129354139625268, 'sr': 0.6038712987483915, 'input_connectivity': 0.12379645556203929, 'rc_connectivity': 0.15107131768020632, 'fb_connectivity': 0.4564053934069279}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.79379162946974}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 2.6839333416104074e-05}}, {'type': 'RLS', 'params': {'output_dim': 1, 'alpha': 0.838800877796766}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 2.9986109681454498e-05}}, {'type': 'RLS', 'params': {'output_dim': 1, 'alpha': 0.2449566053752632}}], 'edges': [[0, 1], [0, 2], [0, 3], [1, 4], [3, 5], [2, 6], [4, 6], [5, 6]]}
inf {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 1.6715727733560162e-05}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.32778680193485976}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.16912171